# Instrument Classification - Part 2 : CNN

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,Activation,BatchNormalization

Using TensorFlow backend.


In [3]:
x_train = np.load('x_trainH.npy')
x_test  = np.load('x_testH.npy')
y_train = np.load('y_trainH.npy')
y_test  = np.load('y_testH.npy')

print("Train Size:", x_train.shape)
print("Test  Size:", x_test.shape)

Train Size: (58655, 96, 87, 1)
Test  Size: (6518, 96, 87, 1)


In [4]:
model = Sequential()

model.add(Conv2D(64,kernel_size=(3,3),padding='same',strides=1,input_shape = x_train[0].shape))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,kernel_size=(3,3),padding='same',strides=1))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256,kernel_size=(3,3),padding='same',strides=1))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(640,kernel_size=(3,3),padding='same',strides=1))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(Dense(4))
model.add(Activation('sigmoid'))

model.summary()

W0803 18:24:09.869329 10660 deprecation_wrapper.py:119] From C:\Users\AgelosK\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0803 18:24:10.235360 10660 deprecation_wrapper.py:119] From C:\Users\AgelosK\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0803 18:24:10.324112 10660 deprecation_wrapper.py:119] From C:\Users\AgelosK\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0803 18:24:10.577468 10660 deprecation_wrapper.py:119] From C:\Users\AgelosK\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0803 18:24:10.580481 10660 deprecation_wrapper.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 87, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 87, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 87, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 43, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 43, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 43, 128)       512       
_________________________________________________________________
activation_2 (Activation)    (None, 48, 43, 128)       0         
__________

In [ ]:
#model = keras.models.load_model('ICmodel_2_mix_big_more.h5')

sgd = keras.optimizers.SGD(lr=0.0001, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
history = model.fit(x_train,y_train,epochs=1,batch_size=32,validation_data=(x_test,y_test))

plt.plot(history.history['acc'], label='Train Accuracy')
plt.plot(history.history['val_acc'], label='Test Accuracy')
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy-Loss')
plt.legend()

model.save('ICmodel_2_mix_big_more.h5')

W0803 18:27:42.002026 10660 deprecation_wrapper.py:119] From C:\Users\AgelosK\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0803 18:27:42.111699 10660 deprecation.py:323] From C:\Users\AgelosK\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 58655 samples, validate on 6518 samples
Epoch 1/1
10976/58655 [====>.........................] - ETA: 1:09:56 - loss: 1.6610 - acc: 0.5804